# Using CorrelationConnection

## Preliminaries

First of all, follow the instructions on README.md to install the software and activate the right Conda enviornment. Once that has been done, you can launch this Jupyter Notebook. 

Load necessary libraries:

In [ ]:
import pickle
import sys
sys.path.append("./bin/")
sys.path.append(".")
import numpy as np
import matplotlib.pyplot as plt

Tell your system where it can find executables:

In [ ]:
sys.path.append("../bin/")
sys.path.append(".")

The line above may change depending on your working directory. The exact way it is written above works if your working directory is a subfolder of ```CorrelationConnection``` (e.g. ```CorrelationConnection/docs```) and ```CorrelationConnection``` contains another sub-folder called ```bin``` containing the files ```exact_diagonalisation_code.py```, ```exact_diagonalisation_code_sparse.py```, etc.

Import some more libraries:

In [ ]:
# import scipy 
# import scipy.sparse.linalg as sprsla
from hamiltonians import NNHamiltonian
from randomizer import RandomizerStateRandomDelta
from stability_analysis_class import StabilityAnalysisSparse
from tools import SijCalculator
# from tools import create_sx_sparse, create_sy_sparse, create_sz_sparse
# from math import sqrt, pi

## Generating data

Let's now get started with the generation of some data from scratch. We will focus on quantum tomography in the ground state as an example.

### Creating a Hamiltonian

To begin, we will create a Hamiltonian object for ```L``` sites with interactions up to next nearest neighbours at temperature ```temp=0```. This is achieved with the class ```NNHamiltonian(L, h, J_onsite, J_nn, J_nnn, temp=0)```. To keep things simple we will make the system a dimer and we will not apply a magnetic field: 

In [ ]:
# 1) SET VALUES FOR INPUTS:

L= 6                                           # <-- Tetramer
h = [[0, 0, 0.5]]                              # <-- Applied field in Z
J_onsite = np.zeros((3, 3))                    # <-- No onsite interactions (superfluous anyway)
J_nnn = np.zeros((3, 3))                       # <-- No nnn interactions
J_nn = [[1.2, 0, 0], [0, 0.8, 0], [0, 0, 1]]   # <-- Some diagonal nn interactions

# 2) CREATE THE HAMILTONIAN: 

temp = 0
HAMILTONIAN = NNHamiltonian(L, h, J_onsite, J_nn, J_nnn, temp=temp)

```NNHamiltonian(L, h, J_onsite, J_nn, J_nnn, temp=0)``` is one of many classes of Hamiltonians.  The others can be found in ```bin/hamiltonians.py```. Here we also set whether the calculations will be for zero or finite temperature states.

Note that the object is not just the Hamiltonian. It includes the temperature as well. The Hamiltonian itself can be extracted using ```get_init_ham()```:

In [ ]:
H=HAMILTONIAN.get_init_ham()

The Hamiltonian ```H``` is not an array, it is a sparse matrix object (from ```scipy.sparse```). Therefore, only the matrix elements that are non zero are stored. If we try to print ```H``` we will get a list of its non-zero elements:

In [ ]:
print(H)

Note the factor of "2" in the definition of the Hamilonian.

Let us now find the ground-state energy and state vector:

In [ ]:
GROUND_STATE=SijCalculator.find_gs_sparse(H)
print("Ground state energy = ",GROUND_STATE[0],"\nGround state vector = \n",np.round(GROUND_STATE[1],2))

Note that we have used np.round to write the grounds state with only 2 decimal points for clarity.

We can also work with a dense-matrix representation:

In [ ]:
H_DENSE=H.todense()

In [ ]:
print(H_DENSE)

```find_eigvals``` can now be used to find all the energies:

In [ ]:
ENERGIES = SijCalculator.find_eigvals(H_DENSE)

In [ ]:
print(ENERGIES)

### Generating fitness landscape data

Now we can carry out the fitness landscape analysis. 

In order to do this we need to set a few parameters that specifiy the type of fitness landscape we want to explore and how.

First, we need to set up a randomizer. There are several available, they are all in ```randomizer.py```. Here we use ```RandomizerStateRandomDelta``` which creates random state vectors the real and imaginary parts of whose amplitudes are picked from a flat probability distribution centred on those of a given input state. 

In [ ]:
delta= 0.5
no_of_processes= 1
distribution_type= {"shape": "normal", "rhoMethod": "squareRandom", "rhoParams": {'min_ev': 0.0}, "scaleWithSize": True}
RANDOMIZER = RandomizerStateRandomDelta(HAMILTONIAN, delta, no_of_processes, distribution_type)

Important parameters here are:
 - ```delta```: a maximum amount for a random scaling factor multiplying any randomly sampled state. The factor itself is generated for every random state from the uniform distribution [0, delta].
 - ```no_of_processes```: number of parallel runs for the generation of random samples. Set this to the amount of threads available for fastest results.
 - ```distribution_type```: a dictionary specifying the type of randomization. You can set up different types of distribution functions (uniform or normal), whether or not there should be a scaling size factor, and in the case of finite temperature calculations different ways of sampling random density matrices $\rho$. 
Now, set up our exploration of the fitness landscape using ```StabilityAnalysisSparse``` from ```stability_analysis_class.py```: 

In [ ]:
corr = ["Sxx", "Sxy", "Sxz", "Syx", "Syy", "Syz", "Szx", "Szy", "Szz"]

In [ ]:
STABILITY_ANALYSIS = StabilityAnalysisSparse(HAMILTONIAN, RANDOMIZER, corr, save_Sqs=True, save_Sijs=True, save_Sqints=True)

Everything is set up to run the algorithm:

In [ ]:
no_of_samples= 50
STABILITY_ANALYSIS.run(no_of_samples)

The output is saved in the ```STABILITY_ANALYSIS``` instance. Let see some relevant examples:

```dist``` contains the distances of the random states to the ground state of the reference Hamiltonian:

In [ ]:
dist = STABILITY_ANALYSIS.dist
dist

```en``` contains the corresponding state energies of the initial Hamiltonian, as compared to the ground state: $\langle \psi_{\text{rand}} | H_{\text{init}} | \psi_{\text{rand}}\rangle - E_0$

In [ ]:
print(STABILITY_ANALYSIS.en)

```diffSij``` contains the distances between real-space correlators:

In [ ]:
STABILITY_ANALYSIS.diffSij

```Sqs``` contains the reciprocal space expressions of all the random correlators, computed on a $100 \times 100$ grid in $\mathbf{q}$-space. In other words, these are the spin-resovled diffuse, magnetic neutron scattering functions. To be more explicit, let 

$\rho_{i,j}^{\alpha,\beta}\left[\psi_n^{\rm rand}\right] 
\equiv 
\left\langle 
    \psi_n^{\rm rand} 
    \left| 
        S_i^{\alpha} S_j^{\beta} 
    \right|
    \psi_n^{\rm rand}
\right\rangle$ 

represent the correlator between the component $\alpha$ of the spin at site $i$ and the component $\beta$ of the spin at site $j$ ($\alpha,\beta=x,y,z$; $i,j=0,1,\ldots,L-1$). We define the reciprocal of this quantity (up to a normalisation constant) as 

$\mathcal{S}_{\alpha,\beta}^n(\mathbf{q})
\equiv 
\frac{1}{L}\sum_{i,j=0}^{L-1}e^{i\mathbf{q}.(\mathbf{R}_i-\mathbf{R}_j)}\rho_{i,j}^{\alpha,\beta}\left[\psi_n^{\rm rand}\right] $

Note that this is not the same as a Fourier transform, as the quantity $\rho_{i,j}^{\alpha,\beta}\left[\psi_n^{\rm rand}\right]$ does not depend only on $\mathbf{R}_i-\mathbf{R}_j$ but in general depends on $\mathbf{R}_i$ and $\mathbf{R}_j$ independently. Indeed, this will be the case for a magnetic molecule sitting somewhere in space, since the structure is not periodic in this case. For the same reason, the vector $\mathbf{q}$ is not restricted to a discretised lattice. It represents the momentum transferred between the sample and the neutron, which can vary continuously. In the code we discretise the values of the two-dimensional vactor $\mathbf{q}=(q_x,q_y)$ by allowing 100 values of $q_x$ and 100 values of $q_y$, each uniformly distributed between $-2\pi$ and $+2\pi$ (giving 10000 values of the wave vector $\mathbf{q}$ in total).

The structure of ```Sqs``` is best illustrated through a couple of  examples. For instance, in our case, 

```Sqs[3]["Sxy"][23][76]``` 

contains the value of $\mathcal{S}_{x,y}^3(\mathbf{q})$ at a wave vector $\mathbf{q}$ given by the coordinates $(23,76)$ on our discretised reciproclal grid:

In [ ]:
Sqs = STABILITY_ANALYSIS.Sqs
print(Sqs[3]["Sxy"][23][76])

Similarly 

```Sqs[2]["Syy"][10][35]``` 

contains the value of  $\mathcal{S}_{y,y}^2(\mathbf{q})$ at a wave vector $\mathbf{q}$ given by the coordinates $(10,35)$:

In [ ]:
print(Sqs[2]["Syy"][10][35])

Note that the $xy$ correlator is a complex number, while the $yy$ correlator is real. This makes sense: $\mathcal{S}_{\alpha,\beta}^n(\mathbf{q})$ is not the expectation value of a Hermitian operator, and in general the sum over $i$ and $j$ will have terms of the form 

$\left\langle 
    \psi_n^{\rm rand} 
    \left|  
    S_i^{\alpha} S_j^{\beta}
 \right|
    \psi_n^{\rm rand}
\right\rangle
e^{i.(\mathbf{R}_i-\mathbf{R}_j)}
+
\left\langle 
    \psi_n^{\rm rand} 
    \left| 
    S_i^{\beta} S_j^{\alpha}
\right|
    \psi_n^{\rm rand}
\right\rangle
e^{-i.(\mathbf{R}_i-\mathbf{R}_j)}$

The two expectation values are both real but in general they are different, and therefore the imaginary parts of the two exponentials do not cancel. For $\alpha=\beta$, they do.

```Sijs``` contains the values of real-space correlators, for instance ```Sijs[3]['Sxz'][0][1]``` is the value of the $xz$ correlator betweent the 1st site and the 2nd site in the 3rd random state, 
$\left\langle 
    \psi_3^{\rm rand} 
    \left| 
        S_0^{x} S_1^{z} 
    \right|
    \psi_3^{\rm rand}
\right\rangle$:

In [ ]:
Sijs = STABILITY_ANALYSIS.Sijs
Sijs[3]['Sxz'][0][1]

```Sqints``` contains the diffuse magnetic neutron scattering functions for each fo the random ground states, computed by combining the individual spin-resolved functions $\mathcal{S}_{\alpha,\beta}\left(\mathbf{q}\right)$ using the formula (up to a normalisation factor)

$    \mathcal{S}(\mathbf{q})
    =
    \sum_{\alpha.\beta}
    \left(
        \delta_{\alpha,\beta}-\hat{\mathbf{q}}_{\alpha}\hat{\mathbf{q}}_{\beta}
    \right)
    \mathcal{S}_{\alpha,\beta}(\mathbf{q}).$
    
Thus for instance ```Sqints[2][10][35]``` is the value of $\mathcal{S}\left(\mathbf{q}\right)$ for the 3rd random state (remember that random states are numbers $n=0,1,2,3,\ldots$) evaluated at the 10th values of $q_x$ and the 35th value of $q_y$:

In [ ]:
Sqints = STABILITY_ANALYSIS.Sqints
Sqints[2][10][35]

Finally, we can calculate the distance between the scattering functions $S'\left(\mathbf{q}\right)$ of a random state and its value $S'\left(\mathbf{q}\right)$ for the ground state of the reference Hamiltonian, evaluated using 

$    \Delta \mathcal{S}=\Delta \mathcal{S}[\mathcal{S}, \mathcal{S}'] = 
    \frac{1}{\mathcal{N}}
    \left\lbrace 
        \int d\mathbf{q} {\left[\mathcal{S}(\mathbf{q}) - \mathcal{S}'(\mathbf{q})\right]}^2
    \right\rbrace^{1/2}.$

With a normalization factor being

$\mathcal{N} = 
    {\left\lbrace
        \int d\mathbf{q} {\left[\mathcal{S}(\mathbf{q}) - \bar{\mathcal{S}}\right]}^2
    \right\rbrace}^{1/2}$

To get differences that are properly normalised, we can use ```return_normalized_diffSqint```:

In [ ]:
Sq_int = STABILITY_ANALYSIS.return_normalized_diffSqint()
Sq_int

We can see that there is a correlation between how close the scattering functions were and how close the states were: 

In [ ]:
fig, ax = plt.subplots()

scale = 20 # size of dots

x, y = dist,Sq_int  
ax.scatter(x, y, c='blue', s=scale, label='A plot',alpha=1.0, edgecolors='none')

min_x=min(x)
max_x=max(x)
del_x=max_x-min_x
min_y=min(y)
max_y=max(y)
del_y=max_y-min_y

plt.xlim(min_x-0.2*del_x,max_x+0.2*del_x)
plt.ylim(min_y-0.2*del_y,max_y+0.2*del_y)
#ax.legend('sss') 

ax.grid(False) # True = grid displayed ; False = no grid displayed

ax.set_aspect(del_x/del_y) # Makes aspect ratio 1:1

ax.axhline(y=0, linewidth=0.5,color='k')
ax.axvline(x=0, linewidth=0.5,color='k')

plt.savefig('filename.png', dpi=300) # Export hires figure

plt.show()

We can see that there's a linear correlation with the scaling coefficient close to 1.

It is illustrative to display the $\mathcal{S}(\mathbf{q})$ for some of the random states (left figures) and compare it to the target scattering function (right figures):

In [ ]:
for n in range(10):
    print(n,"th random state, dist=",dist[n],"Sq_int=",Sq_int[n],":")
    fig, axs = plt.subplots(1, 2)
    axs[0].imshow(Sqints[n])
    axs[1].imshow(STABILITY_ANALYSIS.Sq_int_in)
    plt.show()

Looking at those figures we confirm what is shown in the linear correlation above. The closer the state is to the target state, its scattering funciton more closely resembles the target scattering function.